In [2]:
! pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 MB 3.4 MB/s eta 0:00:00


# Hyperparameter Tuning

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
import joblib
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
unscaled_data = pd.read_csv('/content/unscaled_processed_data.csv')
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled=joblib.load("/content/sacled_train_test_split.pkl")

In [5]:
X_unscaled = unscaled_data.drop(columns='price')
y_unscaled = unscaled_data['price']

In [6]:
X_train_unscaled, X_test_unscaled, y_train_unscaled, y_test_unscaled = train_test_split(X_unscaled, y_unscaled, test_size=0.2, random_state=42)

## Support Vector Regressor (SVR)

In [ ]:
param_grid_svr = {
    'kernel': ['rbf', 'poly'],
    'C': [0.1, 1, 10],
    'epsilon': [0.01, 0.1, 0.5],
    'gamma': ['scale', 'auto']
}

grid_svr = GridSearchCV(SVR(), param_grid_svr, cv=5, scoring='neg_mean_squared_error')
grid_svr.fit(X_train_scaled, y_train_scaled)
print("Best SVR Params:", grid_svr.best_params_)

## Random Forest Regressor

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_rf = GridSearchCV(RandomForestRegressor(), param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X_train_unscaled, y_train_unscaled)
print("Best RF Params:", grid_rf.best_params_)

Best RF Params: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}


## XGBoost Regressor

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 1.0]
}

grid_xgb = GridSearchCV(xgb.XGBRegressor(objective='reg:squarederror'), param_grid_xgb, cv=5, scoring='neg_mean_squared_error')
grid_xgb.fit(X_train_unscaled, y_train_unscaled)
print("Best XGB Params:", grid_xgb.best_params_)

Best XGB Params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 100, 'subsample': 1.0}


## Linear Regression

In [ ]:
model_lr = LinearRegression()
scores = cross_val_score(model_lr, X_train_scaled, y_train_scaled, scoring='neg_mean_squared_error', cv=5)
print("Linear Regression CV RMSE:", (-scores.mean())**0.5)

Linear Regression CV RMSE: 0.1462767119159479


## Ridge Regression

In [ ]:
param_grid_ridge = {
    'alpha': [0.01, 0.1, 1, 10, 100]
}

grid_ridge = GridSearchCV(Ridge(), param_grid_ridge, cv=5, scoring='neg_mean_squared_error')
grid_ridge.fit(X_train_scaled, y_train_scaled)

print("Best Ridge α:", grid_ridge.best_params_['alpha'])
print("Ridge Regression CV RMSE:", (-grid_ridge.best_score_)**0.5)

Best Ridge α: 0.01
Ridge Regression CV RMSE: 0.14627671201186718
